In [1]:
import pandas as pd
import numpy as np

In [9]:
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)


In [10]:
from nltk.tokenize import word_tokenize 
from nltk.tokenize import RegexpTokenizer
from nltk.stem import LancasterStemmer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('english_cleaned_lyrics.csv')

In [3]:
df

,Unnamed: 0,index,song,year,artist,genre,lyrics
0,0,0,ego-remix,2009,beyonce-knowles,Pop,Oh baby how you doing You know I'm gonna cut r...
1,1,1,then-tell-me,2009,beyonce-knowles,Pop,playin everything so easy it's like you seem s...
2,2,2,honesty,2009,beyonce-knowles,Pop,If you search For tenderness It isn't hard to ...
3,3,3,you-are-my-rock,2009,beyonce-knowles,Pop,Oh oh oh I oh oh oh I If I wrote a book about ...
4,4,4,black-culture,2009,beyonce-knowles,Pop,Party the people the people the party it's pop...
...,...,...,...,...,...,...,...
218205,362232,362232,who-am-i-drinking-tonight,2012,edens-edge,Country,I gotta say Boy after only just a couple of da...
218206,362233,362233,liar,2012,edens-edge,Country,I helped you find her diamond ring You made me...
218207,362234,362234,last-supper,2012,edens-edge,Country,Look at the couple in the corner booth Looks a...
218208,362235,362235,christ-alone-live-in-studio,2012,edens-edge,Country,When I fly off this mortal earth And I'm measu...


In [71]:
data = df.to_numpy()

In [73]:
data[0]

array([0, 0, 'ego-remix', 2009, 'beyonce-knowles', 'Pop',
       "Oh baby how you doing You know I'm gonna cut right to the chase Some women were made but me myself I like to think that I was created for a special purpose You know what's more special than you You feel me It's on baby let's get lost You don't need to call into work cause you're the boss For real want you to show me how you feel I consider myself lucky that's a big deal Why Well you got the key to my heart But you ain't gonna need it I'd rather you open up my body And show me secrets you didn't know was inside No need for me to lie It's too big it's too wide It's too strong it won't fit It's too much it's too tough He talk like this cause he can back it up He got a big ego such a huge ego I love his big ego it's too much He walk like this cause he can back it up Usually I'm humble right now I don't choose You can leave with me or you could have the blues Some call it arrogant I call it confident You decide when you find 

In [74]:
x = data[:,6]

In [75]:
x

array(["Oh baby how you doing You know I'm gonna cut right to the chase Some women were made but me myself I like to think that I was created for a special purpose You know what's more special than you You feel me It's on baby let's get lost You don't need to call into work cause you're the boss For real want you to show me how you feel I consider myself lucky that's a big deal Why Well you got the key to my heart But you ain't gonna need it I'd rather you open up my body And show me secrets you didn't know was inside No need for me to lie It's too big it's too wide It's too strong it won't fit It's too much it's too tough He talk like this cause he can back it up He got a big ego such a huge ego I love his big ego it's too much He walk like this cause he can back it up Usually I'm humble right now I don't choose You can leave with me or you could have the blues Some call it arrogant I call it confident You decide when you find on what I'm working with Damn I know I'm killing you with 

In [79]:
y = data[:,5]

In [80]:
y[0]

'Pop'

In [81]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r',encoding='utf-8') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [82]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B.50d.txt')

In [83]:
word_to_index['baseball']

69714

In [84]:
len(df['genre'].unique())

11

In [107]:
def sentences_to_indices(X, word_to_index, max_len):
    
    m = X.shape[0]                                  
    
    X_indices = np.zeros((m, max_len))
    
    for i in range(m):                             
        

        sentence_words = [w.lower() for w in X[i].split()]
        
     
        j = 0
        
      
        for w in sentence_words:
            if j>999:
                break
            if w in word_to_index:
                X_indices[i, j] = word_to_index[w]
            j += 1

    
    return X_indices

In [86]:

X1 = np.array(["univ lol", "lets play baseball", "food is ready for you"])
X1_indices = sentences_to_indices(X1,word_to_index, max_len = 5)
print("X1 =", X1)
print("X1_indices =", X1_indices)

X1 = ['univ lol' 'lets play baseball' 'food is ready for you']
X1_indices = [[372217. 225122.      0.      0.      0.]
 [220930. 286375.  69714.      0.      0.]
 [151204. 192973. 302254. 151349. 394475.]]


In [87]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
   
    vocab_len = len(word_to_index) + 1                 
    emb_dim = word_to_vec_map["cucumber"].shape[0]      
    
  
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]
        
    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    embedding_layer.build((None,))
    
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [88]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])

weights[0][1][3] = -0.3403


In [89]:
def Emojify_V2(input_shape, word_to_vec_map, word_to_index):
  
    
    sentence_indices = Input(input_shape, dtype='int32')
    

    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
   
    embeddings = embedding_layer(sentence_indices)   
    
 
    X = LSTM(128, return_sequences=True)(embeddings)
    
    X = Dropout(0.5)(X)
   
    X = LSTM(128, return_sequences=False)(X)
    
    X = Dropout(0.5)(X)
    
    X = Dense(11)(X)
    
    X = Activation('softmax')(X)
    
    
    model = Model(inputs=sentence_indices, outputs=X)
    

    
    return model

In [90]:
model = Emojify_V2((1000,), word_to_vec_map, word_to_index)
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1000)]            0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 1000, 50)          20000050  
_________________________________________________________________
lstm_2 (LSTM)                (None, 1000, 128)         91648     
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000, 128)         0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 11)               

In [91]:

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [92]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

In [93]:
x.shape

(218210,)

In [94]:

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [95]:
X_train

array(['High low here we go Hear the word That floats upon the air above Reach up touch the night Catch a light That\'s falling on the only ones The only ones And the way you see your life A case of "do or die" Returns us to the way It used to be before And the future lies In the years of sacrifice You\'ll open up your eyes And wake to see it Standing at your door Right time turn it on Touch a heart That\'s waiting for the only one Night time can\'t go wrong Open up and sing the song For everyone And the way you see your life A case of "do or die" To preserve your sanity When others lose control And the years we spent Heaven blessed or heaven sent Sang where kings and presidents Could change the world Just standing at the wall Why tell me why don\'t you know Do you believe in dreams Tell me where did they go Why did they leave us If you ask me I don\'t know Where did the music go After all is said and done Same chance for everyone If you ask us we can\'t say When did the music go away 

In [111]:
def one_hot(array):
    unique, inverse = np.unique(array, return_inverse=True)
    onehot = np.eye(unique.shape[0])[inverse]
    return onehot

In [112]:
X_train_indices = sentences_to_indices(X_train, word_to_index, 1000)
Y_train_oh = one_hot(y_train)

In [113]:
Y_train_oh

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
model.fit(X_train_indices, Y_train_oh, epochs = 10, batch_size = 32, shuffle=True)

Epoch 1/10
4569/4569 [==============================] - 11958s 3s/step - loss: 1.7343 - accuracy: 0.4670
Epoch 2/10
 827/4569 [====>.........................] - ETA: 2:39:21 - loss: 1.6286 - accuracy: 0.5038